# URL clustering based on similarity

- urls_df = full dataset
- urls_tdf = full dataset transformed
- urls_tsdf = sampled dataset transformed

## Imports & Helper functions

### Save or load jupyter session

### Imports

In [1]:
import pickle
import warnings

import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, fcluster, linkage
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp
import seaborn as sns
import smaz
import tldextract
from sklearn.cluster import Birch
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler

warnings.filterwarnings("ignore")

sns.set()


KeyboardInterrupt



### Helper functions

In [ ]:
from polyleven import levenshtein


def levenshtein_pdist(u, v):
    if isinstance(u, np.ndarray):
        u = u[0]
    if isinstance(v, np.ndarray):
        v = v[0]
    return levenshtein(u, v)

## Preprocessing

In [ ]:
urls_df = pd.read_csv(
    "datasets/kaggle_siddharta_malicious_benign.csv",
    delimiter=",",
    dtype={"url": "string"},
)

urls_df.info()

### Extract domain names from URLs

In [ ]:
# Regex pattern to extract fully qualified domain name (FQDN)
pattern = r"(?:.*?:\/\/)?(?P<www>[wW]{3}\.)?(?P<domain>[\w\.\-]+)[^\w]*"

# Execute regex over URLs
match = urls_df["url"].str.extract(pattern)

# Extract domain using named group
urls_df["FQDN"] = match["domain"]

# Indicate if www subdomain is present
urls_df["has_www"] = match["www"].notna()

urls_df.head()

### Remove all addreses without domain (IPs)

In [ ]:
# Pattern that matches all IPv4 addresses
pattern = "(?:.*?:\/\/)?(?P<www>[wW]{3}\.)?[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}([/:].*)?$"

# Leave only data not containing pure IPv4
urls_df = urls_df[~urls_df["url"].str.match(pattern)]

# Reset index
urls_df = urls_df.reset_index(drop=True)

### Extract features from domains

#### Separate TLD, domain and subdomain

In [ ]:
# Function to extract components of domain using tldextract
def extract_domain_components(url):
    ext = tldextract.extract(url)
    return pd.Series([ext.subdomain, ext.domain, ext.suffix, ext.suffix == ""])

In [ ]:
# Apply function to url column to extract domain components and explode into separate columns
urls_df[["subdomain", "domain", "TLD", "is_invalid_TLD"]] = urls_df["url"].apply(
    extract_domain_components
)

In [ ]:
urls_df[urls_df["is_invalid_TLD"]].head()

In [ ]:
# Remove domains with invalid TLD
urls_df = urls_df[~urls_df["is_invalid_TLD"]]

# Reset index
urls_df = urls_df.reset_index(drop=True)

#### Length of domain, subdomain and TLD

In [ ]:
urls_df[["domain_length", "subdomain_length", "TLD_length"]] = urls_df[
    ["domain", "subdomain", "TLD"]
].applymap(len)
urls_df.head(2)

#### Number of subdomains

I decide to include www in the count of subdomains. Might make performance-wise issues later. Reconsider if needed

In [ ]:
urls_df["num_of_subdomains"] = (
    urls_df["subdomain"].str.split(".").apply(lambda x: len(x) if x != [""] else 0)
)
urls_df.head(2)

#### Characters frequency & vowel-to-consonant ratio
Characters:
- alphabetical - "a-zA-Z"
- digits - "0-9"
- special - all except alphabetical, digits and dot

Can be changed based on the occurences of dots. It may be better to remove dots so this information is uncorrelated with num_of_substrings

Try the result with and without dots to analyze the difference

In [ ]:
for column in ["domain", "subdomain", "TLD"]:
    # Vowel-to-consonant ratio
    vowel_counts = urls_df[column].str.count(r"[aeiouAEIOU]")
    consonant_counts = urls_df[column].str.count(r"[b-df-hj-np-tv-zB-DF-HJ-NP-TV-Z]")

    # Get alphabetical, numeric and special character counts for specific column
    numeric_counts = urls_df[column].str.count(r"[0-9]")
    special_counts = urls_df[column].str.count(r"[^A-Za-z0-9\s\.]")
    alpha_counts = vowel_counts + consonant_counts

    # Add them into DF
    urls_df[
        [
            f"{column}_alpha_count",
            f"{column}_numeric_count",
            f"{column}_special_count",
            f"{column}_vowel_consonant_ratio",
        ]
    ] = pd.Series(
        [alpha_counts, numeric_counts, special_counts, vowel_counts / consonant_counts]
    )

urls_df.head(2)

#### Complexity of domain and subdomain

Using compression algorithm (`smaz` python implementation) to approximate Kolmogorov complexity

In [ ]:
urls_df[["domain_complexity", "subdomain_complexity"]] = urls_df[
    ["domain", "subdomain"]
].applymap(lambda s: len(smaz.compress(s)) / len(s) if s != "" else np.nan)

### Encode labels

In [ ]:
# Create OneHotEncoded features from type


ohenc = OneHotEncoder(sparse_output=False)
type_ohenc = pd.DataFrame(
    ohenc.fit_transform(urls_df["type"].values.reshape(-1, 1)),
    columns=ohenc.categories_[0],
).astype(bool)

# URLs_transformed df
urls_tdf = pd.concat([urls_df, type_ohenc], axis=1)

In [ ]:
urls_tdf["malicious"] = ~urls_tdf["benign"]
urls_tdf.head()

### TF-IDF

#### Incremental wrapper

In [ ]:
class IncrementalTfidf:
    """
    A class to process text data in chunks and incrementally compute character-level TF-IDF values.

    Attributes:
    -----------
    hashing_vectorizer : HashingVectorizer
        Vectorizer that converts text data to a term-document matrix using the hashing trick
    tfidf_transformer : TfidfTransformer
        Transformer that computes TF-IDF values from the term-document matrix
    X_counts : scipy.sparse matrix
        Accumulated term-document matrix
    X_tfidf : scipy.sparse matrix
        Accumulated TF-IDF representation
    """

    def __init__(
        self,
        ngram_range=(1, 1),
        n_features=2**20,
    ):  # Default value for n_features in HashingVectorizer
        """
        Initializes the IncrementalTfidf with a HashingVectorizer and TfidfTransformer.
        """
        self.hashing_vectorizer = HashingVectorizer(
            analyzer="char",
            token_pattern=None,
            n_features=n_features,
            ngram_range=ngram_range,
        )
        self.tfidf_transformer = TfidfTransformer()
        self.X_counts = None
        self.X_tfidf = None

    def update_tf_counts(self, chunk):
        """
        Updates the term-document matrix with the new chunk of text data.

        Parameters:
        -----------
        chunk : pandas.Series or list of str
            New chunk of text data
        """
        # Transform the chunk of text data into a term-document matrix using the HashingVectorizer
        chunk_counts = self.hashing_vectorizer.transform(chunk)

        # If this is the first chunk, set the term-document matrix to the transformed chunk
        if self.X_counts is None:
            self.X_counts = chunk_counts
        else:
            # Otherwise, stack the transformed chunk to the existing term-document matrix
            self.X_counts = sp.vstack((self.X_counts, chunk_counts))

    def update_idf(self):
        """
        Updates the TfidfTransformer based on the current term-document matrix.
        """
        self.tfidf_transformer.fit(self.X_counts)

    def partial_fit(self, chunk):
        """
        Updates the term-document matrix and fits the TfidfTransformer with the new chunk of text data.

        Parameters:
        -----------
        chunk : pandas.Series or list of str
            New chunk of text data

        Returns:
        --------
        self : IncrementalTfidf
            The instance of the IncrementalTfidf
        """
        self.update_tf_counts(chunk)
        self.update_idf()
        return self

    def transform(self, chunk):
        """
        Transforms the given chunk of text data to a TF-IDF representation.

        Parameters:
        -----------
        chunk : pandas.Series or list of str
            Chunk of text data to be transformed

        Returns:
        --------
        chunk_tfidf : scipy.sparse matrix
            Transformed chunk in TF-IDF representation
        """
        chunk_counts = self.hashing_vectorizer.transform(chunk)
        chunk_tfidf = self.tfidf_transformer.transform(chunk_counts)
        return chunk_tfidf

    def partial_fit_transform(self, chunk):
        """
        Updates the term-document matrix, fits the TfidfTransformer, and transforms the given chunk of text data.

        Parameters:
        -----------
        chunk : pandas.Series or list of str
            New chunk of text data

        Returns:
        --------
        chunk_tfidf : scipy.sparse matrix
            Transformed chunk in TF-IDF representation
        """
        self.partial_fit(chunk)
        return self.transform(chunk)

    def compute_tfidf(self):
        """
        Retrieves the accumulated TF-IDF representation of the processed text data.

        Returns:
        --------
        X_tfidf : scipy.sparse matrix
            Accumulated TF-IDF representation
        """
        self.X_tfidf = self.tfidf_transformer.transform(self.X_counts)
        return self.X_tfidf

#### The code

In [ ]:
NGRAM_RANGE = (1, 5)

In [ ]:
incremental_domain_tfidf = IncrementalTfidf(ngram_range=NGRAM_RANGE)
incremental_subdomain_tfidf = IncrementalTfidf(ngram_range=NGRAM_RANGE)

incremental_domain_tfidf.partial_fit(urls_df["domain"])
incremental_subdomain_tfidf.partial_fit(urls_df["subdomain"])

# Retrieve the accumulated TF-IDF representation
X_domain_tfidf = incremental_domain_tfidf.compute_tfidf()
X_subdomain_tfidf = incremental_subdomain_tfidf.compute_tfidf()

### Feature selection & scaling

#### TF-IDF selection parameter tuning

In [ ]:
# Open the file in binary mode
with open("domain_n_components_search.pkl", "rb") as file:
    components_variance_time_d = pickle.load(file)

with open("subdomain_n_components_search.pkl", "rb") as file:
    components_variance_time_s = pickle.load(file)

In [ ]:
components_variance_time_d

In [ ]:
components_variance_time_s

In [ ]:
# Plot the cumulative explained variance ratio as a function of the number of components
plt.figure(figsize=(10, 6))
plt.plot(
    components_variance_time_d["n_components"],
    components_variance_time_d["explained_variance"],
    marker="o",
)
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance Ratio")
plt.title("Explained Variance Ratio vs Number of Components - Domains")
plt.show()

In [ ]:
# Plot the cumulative explained variance ratio as a function of the number of components
plt.figure(figsize=(10, 6))
plt.plot(
    components_variance_time_s["n_components"],
    components_variance_time_s["explained_variance"],
    marker="o",
)
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance Ratio")
plt.title("Explained Variance Ratio vs Number of Components - Subdomains")
plt.show()

In [ ]:
# Open the file in binary mode
with open("domain_n_components_search.pkl", "wb") as file:
    pickle.dump(components_variance_time_d, file)

with open("subdomain_n_components_search.pkl", "wb") as file:
    pickle.dump(components_variance_time_s, file)

#### Non-TF-IDF feature imputation, scaling & selection

In [ ]:
urls_df_imputed = urls_df.copy()

# Replace NaN values with zeros
urls_df_imputed.fillna(0, inplace=True)

# Replace infinities with the maximum finite value in each column
urls_df_imputed = urls_df_imputed.replace([np.inf, -np.inf], np.nan)
max_values = urls_df_imputed.max()
urls_df_imputed.fillna(max_values, inplace=True)

In [ ]:
ssc = StandardScaler()

urls_df_relevant_scaled = ssc.fit_transform(urls_df_imputed.iloc[:, 8:])

In [ ]:
pca = PCA(n_components=0.95)
urls_df_relevant_selected = pca.fit_transform(urls_df_relevant_scaled)

urls_df_relevant_sparse = sp.csr_matrix(urls_df_relevant_selected)

#### TF-IDF feature selection

In [ ]:
# Taken from https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#performing-dimensionality-reduction-using-lsa

lsa_domain = make_pipeline(TruncatedSVD(n_components=350), Normalizer(copy=False))
lsa_subdomain = make_pipeline(TruncatedSVD(n_components=300), Normalizer(copy=False))

X_domain_lsa = lsa_domain.fit_transform(X_domain_tfidf)
explained_variance_d = lsa_domain[0].explained_variance_ratio_.sum()

X_subdomain_lsa = lsa_subdomain.fit_transform(X_subdomain_tfidf)
explained_variance_s = lsa_subdomain[0].explained_variance_ratio_.sum()

print(
    f"Explained variance of domain in the SVD step: {explained_variance_d * 100:.1f}%"
)
print(
    f"Explained variance of subdomain in the SVD step: {explained_variance_s * 100:.1f}%"
)

In [ ]:
X = sp.hstack((urls_df_relevant_sparse, X_domain_lsa, X_subdomain_lsa), format="csr")

## Model training

In [ ]:
SAMPLE_SIZE = 40000

np.random.seed(42)

random_indices = np.random.choice(X.shape[0], size=SAMPLE_SIZE, replace=False)

X_subset = X[random_indices]

In [ ]:
# Create a BIRCH clustering instance
birch_clustering = Birch(n_clusters=None)

# Fit the BIRCH clustering model on the TF-IDF data
birch_clustering.fit(X_subset)

# Get the cluster labels for each data point
labels = birch_clustering.labels_

## Model evaluation

In [ ]:
plt.plot(np.bincount(labels))

In [ ]:
SAMPLE_SIZE = 5000

np.random.seed(24)

random_indices = np.random.choice(
    X_subset.shape[0], size=SAMPLE_SIZE, replace=False
)  # Slice the sparse matrix using the generated indices


X_sample_sparse = X_subset[random_indices]
labels_sample = labels[random_indices]

In [ ]:
# Calculate Silhouette Score
sil_score = silhouette_score(X_sample_sparse, labels_sample)

# Calculate Calinski-Harabasz Index
X_sample_dense = X_sample_sparse.toarray()
ch_index = calinski_harabasz_score(X_sample_dense, labels_sample)

# Calculate Davies-Bouldin Index
db_index = davies_bouldin_score(X_sample_dense, labels_sample)

# Print the scores
print("Silhouette Score:", sil_score)
print("Calinski-Harabasz Index:", ch_index)
print("Davies-Bouldin Index:", db_index)

In [ ]:
len([node for node in birch_clustering.subcluster_centers_])

## Data exploration

### Levenshtein distances

#### Domain unchanged

In [ ]:
# Create two sets of N_SAMPLES random samples
N_SAMPLES = 50000

urls_tsdf = pd.DataFrame()

urls_tsdf["sample1"] = urls_tdf.sample(n=N_SAMPLES, random_state=123).reset_index(
    drop=True
)["domain"]
urls_tsdf["sample2"] = urls_tdf.sample(n=N_SAMPLES, random_state=545).reset_index(
    drop=True
)["domain"]

In [ ]:
# Calculate Levenshtein distance on each pair (N_SAMPLES distances)
urls_tsdf["levenshtein_distance_domain"] = urls_tsdf.apply(
    lambda row: levenshtein(row.sample1, row.sample2), axis=1
)

In [ ]:
urls_tsdf.describe()

In [ ]:
urls_tsdf

In [ ]:
sns.displot(
    urls_tsdf,
    x="levenshtein_distance_domain",
    binwidth=3,
    height=10,
)
plt.title(
    f"Distribution of levenshtein's distances among domains over {N_SAMPLES} random samples"
)

In [ ]:
sns.displot(
    urls_tsdf[urls_tsdf["levenshtein_distance_domain"] < 40],
    x="levenshtein_distance_domain",
    binwidth=1,
    height=8,
)
plt.title(
    f"Distribution of levenshtein's distances among domains over {N_SAMPLES} random samples"
)

## Experiments

### Unchanged URLs

Keep `urls_tdf` intact for this section to show what it should

#### DBSCAN

DBSCAN will not work. It needs $\mathcal{O}(n^2)$

#### Hierarchical clustering

In [ ]:
# Create two sets of N_SAMPLES random samples
N_SAMPLES = 1000

urls_tsdf = urls_tdf.sample(n=N_SAMPLES, random_state=111).reset_index(drop=True)

In [ ]:
# Calculate pairwise distances between domains using levenshtein distance function
X = urls_tsdf["domain"].values.reshape(-1, 1)
distances = pdist(X, metric=levenshtein_pdist)
distances_squareform = squareform(distances)

In [ ]:
# Perform hierarchical clustering
Z = linkage(distances, "complete")

# Draw dendrogram for visual cutoff selection
fig, ax = plt.subplots(figsize=(40, 20))
dendrogram(Z, ax=ax)


fig1 = fig
plt.show()

In [ ]:
CUTOFF = [42, 23, 19, 31]

# Load the figure object from the previous cell
fig = fig1

# Get the axes object from the figure
ax = fig.axes[0]

# Add the cutoff horizontal line
for cutoff in CUTOFF:
    ax.axhline(y=cutoff, color="r", linestyle="--")
    ax.text(x=ax.get_xlim()[0], y=cutoff, s=f"Cutoff: {cutoff}", va="center")


# Show the plot
fig

In [ ]:
# Determine the optimal number of clusters
max_d = 15  # set the threshold distance
clusters = fcluster(Z, max_d, criterion="distance")

In [ ]:
# Add cluster labels to the original dataset
urls_tsdf["cluster"] = clusters

#### Evaluation of cluster quality based on cutoff

In [ ]:
from sklearn.metrics import silhouette_score

# Compute the silhouette score
silhouette_avg = silhouette_score(
    distances_squareform, urls_tsdf["cluster"], metric="precomputed"
)
print(f"Silhouette score: {silhouette_avg}")

# calculate prevalence of malicious domains in each cluster
cluster_prevalence = urls_tsdf.groupby("cluster")["malicious"].mean()

# group by cluster id and count the number of items in each cluster
cluster_counts = urls_tsdf.groupby("cluster").count()["url"]

# create a dataframe combining the cluster counts and cluster prevalence
cluster_data = pd.DataFrame({"count": cluster_counts, "prevalence": cluster_prevalence})

# filter perfect clusters
non_trivial_clusters = cluster_data.loc[
    (cluster_data["prevalence"] != 0) & (cluster_data["prevalence"] != 1)
]

print(f"Total count of samples {len(urls_tsdf)}")
print(f"Total count of clusters {len(cluster_data)}")
print(
    f"Count of samples in perfect clusters {len(urls_tsdf) - non_trivial_clusters['count'].sum()}"
)

print(
    f"Prevalence of non-perfect malicious domains within clusters:\n{non_trivial_clusters}"
)

In [ ]:
# reset the index to get the cluster id as a column
cluster_data = cluster_data.reset_index()

# Create color palette
colors = sns.color_palette("viridis", as_cmap=True)

# Create bar plot
fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(
    x="cluster",
    y="prevalence",
    data=cluster_data,
    palette=colors(cluster_data["count"] / cluster_data["count"].max()),
    ax=ax,
    dodge=False,
)

# Set labels and title
ax.set_title("Prevalence of Malicious Domains by Cluster")
ax.set_xlabel("Cluster Number")
ax.set_ylabel("Prevalence")

# Move the legend outside the plot and make it a gradient line
sm = plt.cm.ScalarMappable(
    cmap=colors, norm=plt.Normalize(vmin=0, vmax=cluster_data["count"].max())
)
sm.set_array([])
cbar = plt.colorbar(
    sm,
    orientation="horizontal",
    pad=0.1,
    shrink=0.5,
    aspect=15,
)
cbar.ax.set_xlabel("Cluster Size")

plt.subplots_adjust(right=0.8)

plt.tight_layout()

plt.show()

In [ ]:
urls_tsdf.info()

In [ ]:
urls_tsdf.groupby("cluster").count()